In [1]:
import sys,os
sys.path.append("..")
import django
django.setup()
import pandas as pd
from io import StringIO
import requests
import datetime
import json
import numpy as np
from crawlers.finlab.import_tools import *

In [20]:
class CrawlStockDivideRatioTW:
    def __init__(self):
        self.target_name = "台股股價還原表"
        self.format = "time_series"
        self.year= datetime.datetime.now().year

    @staticmethod
    def otc_date():
        y = datetime.datetime.now().year
        m = datetime.datetime.now().month
        d = datetime.datetime.now().day
        y = str(y - 1911)
        m = str(m) if m > 9 else '0' + str(m)
        d = str(d) if d > 9 else '0' + str(d)
        datestr = '%s/%s/%s' % (y, m, d)
        return datestr

    def twse_divide_ratio(self):
        datestr = datetime.datetime.now().strftime('%Y%m%d')
        res = requests.get(
            "https://www.twse.com.tw/exchangeReport/TWT49U?response=csv&strDate="+'2004'+"0101&endDate=" \
            + datestr + "&_=1651532565786")
        df = pd.read_csv(StringIO(res.text.replace("=", "")), header=1)
        df = df.dropna(thresh=5).dropna(how='all', axis=1)
        df = df[~df['資料日期'].isnull()]

        # set stock id
        df['stock_id'] = df['股票代號']

        # set dates
        df = df[~df['資料日期'].isnull()]
        years = df['資料日期'].str.split('年').str[0].astype(int) + 1911
#         years.loc[df['資料日期'].str[3] != '年'] = np.nan
        years.loc[years > datetime.datetime.now().year] = np.nan
        years.ffill(inplace=True)
        dates = years.astype(int).astype(str) + '/' + df['資料日期'].str.split('年').str[1].str.replace('月','/').str.replace('日','')
                                                                                                                                                                                                        
        df['date'] = pd.to_datetime(dates, errors='coerce')

        # convert to float
        float_name_list = ['除權息前收盤價', '除權息參考價', '權值+息值', '漲停價格',
                           '跌停價格', '開盤競價基準', '減除股利參考價', '最近一次申報每股 (單位)淨值',
                           '最近一次申報每股 (單位)盈餘']

        df[float_name_list] = df[float_name_list].astype(str).apply(lambda s: s.str.replace(',', '')).astype(float)
        df['divide_ratio'] = df['除權息前收盤價'] / df['開盤競價基準']
        df = df.drop(columns=['資料日期', '股票代號', '漲停價格', '跌停價格', '詳細資料', '最近一次申報資料 季別/日期',
                              '最近一次申報每股 (單位)淨值', '最近一次申報每股 (單位)盈餘', '減除股利參考價'])
        df.columns = ['stock_name', 'divide_before', 'divide_after', 'divide_value', 'divide_category', 'divide_open',
                      'stock_id', 'date', 'divide_ratio']
        return df

    def otc_divide_ratio(self):
        res_otc = requests.get(
            'https://www.tpex.org.tw/web/stock/exright/dailyquo/exDailyQ_result.php?l=zh-tw&d='+str(self.year-1911)+'/01/01&ed='
            + self.otc_date() + '&_=1651594269115')
        df = pd.DataFrame(json.loads(res_otc.text)['aaData'])
        df.columns = ['除權息日期', '代號', '名稱', '除權息前收盤價', '除權息參考價',
                      '權值', '息值', "權+息值", "權/息", "漲停價格",
                      "跌停價格", "開盤競價基準", "減除股利參考價", "現金股利", "每千股無償配股", 
                      "現金增資股數", "現金增資認購價","公開承銷股數", "員工認購股數", "原股東認購數", "按持股比例千股認購"]


        float_name_list = ['除權息前收盤價', '除權息參考價',
                           '權值', '息值', "權+息值", "漲停價格", "跌停價格", "開盤競價基準",
                           "減除股利參考價", "現金股利", "每千股無償配股", "現金增資股數", "現金增資認購價",
                           "公開承銷股數", "員工認購股數", "原股東認購數", "按持股比例千股認購"
                           ]
        df[float_name_list] = df[float_name_list].astype(str).apply(lambda s: s.str.replace(',', '')).astype(float)

        # set stock id
        df['stock_id'] = df['代號']

        # set dates
        dates = df['除權息日期'].str.split('/')
        dates = (dates.str[0].astype(int) + 1911).astype(str) + '/' + dates.str[1] + '/' + dates.str[2]
        df['date'] = pd.to_datetime(dates)
        df['divide_ratio'] = df['除權息前收盤價'] / df['開盤競價基準']
        df = df.drop(columns=['除權息日期', '代號', '權值', '息值', '漲停價格', '跌停價格', '減除股利參考價', '現金股利',
                              '每千股無償配股', '現金增資股數',
                              '現金增資認購價', '公開承銷股數', '員工認購股數', '原股東認購數', '按持股比例千股認購'])
        df.columns = ['stock_name', 'divide_before', 'divide_after', 'divide_value', 'divide_category', 'divide_open',
                      'stock_id', 'date', 'divide_ratio']
        return df

    def twse_cap_reduction(self):
        datestr = datetime.datetime.now().strftime('%Y%m%d')
        res3 = requests.get(
            "https://www.twse.com.tw/exchangeReport/TWTAUU?response=csv&strDate="+str(self.year)+"0101&endDate="
            + datestr + "&_=1651597854043")
        df = pd.read_csv(StringIO(res3.text), header=1)
        df = df.dropna(thresh=5).dropna(how='all', axis=1)
        dates = (df['恢復買賣日期'].str.split('/').str[0].astype(int) + 1911).astype(str) + df['恢復買賣日期'].str[3:]
        df['date'] = pd.to_datetime(dates, errors='coerce')
        df['stock_id'] = df['股票代號'].astype(str)
        df['開盤競價基準']=[a if a>0 else b for a,b in zip(df['開盤競價基準'],df['恢復買賣參考價'])]
        df['divide_ratio'] = df['停止買賣前收盤價格'] / df['開盤競價基準']
        df = df.drop(columns=['恢復買賣日期', '股票代號', '漲停價格', '跌停價格', '詳細資料'])
        df.columns = ['stock_name', 'divide_before', 'divide_after', 'divide_open', 'divide_value', 'divide_category',
                      'date', 'stock_id', 'divide_ratio']
        df['divide_value'] = None
        return df

    def otc_cap_reduction(self):
        res4 = requests.get(
            "https://www.tpex.org.tw/web/stock/exright/revivt/revivt_result.php?l=zh-tw&d="+str(self.year-1911)+"/01/01&ed="
            + self.otc_date() + "&_=1651611342446")
        df = pd.DataFrame(json.loads(res4.text)['aaData'])
        name = ['恢復買賣日期', '股票代號', '股票名稱', '最後交易之收盤價格',
                '減資恢復買賣開始日參考價格', '漲停價格', '跌停價格', '開始交易基準價', '除權參考價', '減資源因', '詳細資料']

        float_name_list = ['最後交易之收盤價格', '減資恢復買賣開始日參考價格', '漲停價格', '跌停價格', '開始交易基準價', '除權參考價']
        df.columns = name
        df[float_name_list] = df[float_name_list].astype(str).apply(lambda s: s.str.replace(',', '')).astype(float)
        df['stock_id'] = df['股票代號'].astype(str)
        dates = (df['恢復買賣日期'].astype(str).str[:-4].astype(int) + 1911).astype(str) + \
                df['恢復買賣日期'].astype(str).str[-4:]
        df['date'] = pd.to_datetime(dates, errors='coerce')
        df['開始交易基準價']=[a if a>0 else b for a,b in zip(df['開始交易基準價'],df['減資恢復買賣開始日參考價格'])]
        df['divide_ratio'] = df['最後交易之收盤價格'] / df['開始交易基準價']
        df = df.drop(columns=['恢復買賣日期', '股票代號', '漲停價格', '跌停價格', '詳細資料'])
        df.columns = ['stock_name', 'divide_before', 'divide_after', 'divide_open', 'divide_value', 'divide_category',
                      'stock_id', 'date', 'divide_ratio']
        df['divide_value'] = None
        return df

    def crawl_main(self):
        df = pd.concat(
            [self.twse_divide_ratio(), self.otc_divide_ratio(), self.twse_cap_reduction(),
             self.otc_cap_reduction()], sort=False)
        df=df[df['divide_ratio']>0]
        return df
    
    
df=CrawlStockDivideRatioTW().twse_divide_ratio()
df

,stock_name,divide_before,divide_after,divide_value,divide_category,divide_open,stock_id,date,divide_ratio
0,佳格,9.30,8.80,0.500000,息,8.80,1227,2004-01-13,1.056818
1,安泰銀,11.25,11.03,0.220000,權,11.25,2849,2004-02-11,1.000000
2,泰金寶-DR,38.00,36.73,1.270000,息,36.70,9105,2004-03-10,1.035422
3,杏輝,19.20,18.91,0.290000,權,19.20,1734,2004-03-12,1.000000
4,萬宇科,37.90,36.70,1.200000,息,36.70,9104,2004-03-19,1.032698
...,...,...,...,...,...,...,...,...,...
11025,凌華,85.20,83.60,1.600000,息,83.60,6166,2020-07-14,1.019139
11026,訊芯-KY,128.50,124.83,3.669145,息,125.00,6451,2020-07-14,1.028000
11027,GIS-KY,117.50,114.00,3.500000,息,114.00,6456,2020-07-14,1.030702
11028,鈺齊-KY,106.50,100.98,5.518897,息,101.00,9802,2020-07-14,1.054455


In [9]:
df2=df.copy()
years = df2['資料日期'].str.split('年').str[0].astype(int) + 1911
years.loc[df2['資料日期'].str[3] != '年'] = np.nan
years.loc[years > datetime.datetime.now().year] = np.nan
years.ffill(inplace=True)
dates = years.astype(int).astype(str) + '/' + df2['資料日期'].str.split('年').str[1].str.replace('月','/').str.replace('日','')  


ValueError: Cannot convert non-finite values (NA or inf) to integer

In [14]:
years.head(200)

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
195   NaN
196   NaN
197   NaN
198   NaN
199   NaN
Name: 資料日期, Length: 200, dtype: float64

In [13]:
df2

,資料日期,股票代號,股票名稱,除權息前收盤價,除權息參考價,權值+息值,權/息,漲停價格,跌停價格,開盤競價基準,減除股利參考價,詳細資料,最近一次申報資料 季別/日期,最近一次申報每股 (單位)淨值,最近一次申報每股 (單位)盈餘,stock_id
0,93年01月13日,1227,佳格,9.30,8.80,0.500000,息,9.40,8.20,8.80,8.80,除權息資料,109年第1季,18.90,0.67,1227
1,93年02月11日,2849,安泰銀,11.25,11.03,0.220000,權,12.00,10.30,11.25,11.25,除權息資料,109年第1季,17.59,0.36,2849
2,93年03月10日,9105,泰金寶-DR,38.00,36.73,1.270000,息,39.30,34.20,36.70,36.73,除權息資料,93年第4季,0.00,4.81,9105
3,93年03月12日,1734,杏輝,19.20,18.91,0.290000,權,20.50,17.60,19.20,19.20,除權息資料,109年第1季,16.71,0.16,1734
4,93年03月19日,9104,萬宇科,37.90,36.70,1.200000,息,39.20,34.20,36.70,36.70,除權息資料,93年第4季,0.00,2.73,9104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11025,109年07月14日,6166,凌華,85.20,83.60,1.600000,息,91.90,75.30,83.60,83.60,除權息資料,109年第1季,21.90,0.40,6166
11026,109年07月14日,6451,訊芯-KY,128.50,124.83,3.669145,息,137.00,112.50,125.00,124.83,除權息資料,109年第1季,50.06,0.46,6451
11027,109年07月14日,6456,GIS-KY,117.50,114.00,3.500000,息,125.00,103.00,114.00,114.00,除權息資料,109年第1季,75.47,0.31,6456
11028,109年07月14日,9802,鈺齊-KY,106.50,100.98,5.518897,息,111.00,90.90,101.00,100.98,除權息資料,109年第1季,48.77,2.02,9802


In [17]:
years = df2['資料日期'].str.split('年').str[0].astype(int) + 1911
# years.loc[df2['資料日期'].str[3] != '年'] = np.nan
years.loc[years > datetime.datetime.now().year] = np.nan
# years.ffill(inplace=True)
years

0        2004.0
1        2004.0
2        2004.0
3        2004.0
4        2004.0
          ...  
11025    2020.0
11026    2020.0
11027    2020.0
11028    2020.0
11029    2020.0
Name: 資料日期, Length: 11030, dtype: float64